In [ ]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

In [ ]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

In [ ]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

In [15]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [16]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"

ouput_file="galaxy_outputs/${filename}_count.csv"  
format=""


In [ ]:
#### Variables
# variants BOOLEEN -v
variants=$(jq -r '.variants' galaxy_inputs/galaxy_inputs.json)
if [[ "$vairants" == "true" ]]; then
    chimera="-v"
    echo "variable variants présente"
else 
    variants=""
fi

# reads BOOLEEN -r
reads=$(jq -r '.reads' galaxy_inputs/galaxy_inputs.json)
if [[ "$reads" == "true" ]]; then
    reads="-r"
    echo "variable reads présente"
else
    reads=""
fi

#symbols BOOLEEN -s
symbols=$(jq -r '.symbols' galaxy_inputs/galaxy_inputs.json)
if [[ "$symbols" == "true" ]]; then
    symbols="-s"
    echo "variable symbols présente"
else
    symbols=""
fi  


####


In [ ]:
./obitools4/build/obicount $variants $reads $symbols $input_file